In [2]:
from scipy.stats import norm
import sqlalchemy as sql
import pandas as pd
import numpy as np
import json

with open("../config.json", "r") as file:
    CONFIG = json.loads(file.read())

greek_cols = [
	'delta',
	'gamma',
	'theta',
	'vega',
	'rho',
	'vanna',
	'vomma',
	'charm',
	'veta',
	'speed',
	'zomma',
	'color',
	'ultima'
]

### Engine

In [3]:
engine = sql.create_engine(CONFIG['db_address'],
                           pool_size=3,
                           max_overflow=0,
                           pool_recycle=299,
                           pool_pre_ping=True)

### TSLA Options Data

In [4]:
query = """
    SELECT
        options.*,
        trm.rate,
        ohlc.adjclose_price AS stock_price,
        ohlc.dividend_yield AS dividend_yield
    FROM
        options
    INNER JOIN
        treasuryratemap AS trm
        USING(date_current, days_to_expiry)
        INNER JOIN
            ohlc
            USING(date_current, ticker)
    WHERE
        ticker = "SPY"
    AND date_current = "2020-11-17"
"""
options = pd.read_sql(query, engine)

ProgrammingError: (MySQLdb._exceptions.ProgrammingError) (1146, "Table 'compour9_test.options' doesn't exist")
[SQL: 
    SELECT
        options.*,
        trm.rate,
        ohlc.adjclose_price AS stock_price,
        ohlc.dividend_yield AS dividend_yield
    FROM
        options
    INNER JOIN
        treasuryratemap AS trm
        USING(date_current, days_to_expiry)
        INNER JOIN
            ohlc
            USING(date_current, ticker)
    WHERE
        ticker = "SPY"
    AND date_current = "2020-11-17"
]
(Background on this error at: http://sqlalche.me/e/f405)

### Calculate Greeks

In [7]:
def calculate_greeks(options):

	if len(options) == 0:
		return options

	o = options.copy()
	m = o.option_type.map({"C" : 1, "P" : -1}).values

	tau = o.days_to_expiry.values / 365
	rtau = np.sqrt(tau)
	iv = o.implied_volatility.values / 100
	S = o.stock_price.values
	K = o.strike_price.values
	q = o.dividend_yield.values / 100
	r = o.rate.values / 100

	###################################################################################################

	eqt = np.exp(-q * tau)
	kert = K * np.exp(-r * tau)

	d1 = np.log(S / K)
	d1 += (r - q + 0.5 * (iv ** 2)) * tau
	d1 /= iv * rtau
	d2 = d1 - iv * rtau

	npd1 = norm.pdf(d1)
	ncd1 = norm.cdf(m * d1)
	ncd2 = norm.cdf(m * d2)

	###################################################################################################

	delta = m * eqt * ncd1

	gamma = np.exp(q - r) * npd1
	gamma /= (S * iv * rtau)

	vega = S * eqt * npd1 * rtau	
	vega /= 100

	rho = m * tau * kert * ncd2
	rho /= 100

	theta = (S * norm.pdf(m * d1) * iv)
	theta *= -eqt / (2 * rtau)
	theta -= m * r * kert * ncd2
	theta += m * q * S * eqt * ncd1
	theta /= 365

	###################################################################################################

	vanna = (vega / S)
	vanna *= (1 - d1 / (iv * rtau))

	vomma = (vega / iv) * (d1 * d2)

	charm = 2 * (r - q) * tau - d2 * iv * rtau
	charm /= 2 * tau * iv * rtau
	charm *= eqt * npd1
	charm = m * q * eqt * ncd1 - charm
	charm /= 365

	veta = q.copy()
	veta += ((r - q) * d1) / (iv * rtau)
	veta -= (1 + d1 * d2) / (2 * tau)
	veta *= -S * eqt * npd1 * rtau
	veta /= 365 * 100

	speed = 1
	speed += d1 / (iv * rtau)
	speed *= -gamma / S

	zomma = (d1 * d2 - 1) / iv
	zomma *= gamma

	color = 2 * (r - q) * tau
	color -= d2 * iv * rtau
	color *= d1 / (iv * rtau)
	color += 2 * q * tau + 1
	color *= -eqt * npd1 / (2 * S * tau * iv * rtau)
	color /= 365

	ultima = d1 * d2 * (1 - d1 * d2) + d1 * d1 + d2 * d2
	ultima *= -vega / (iv * iv)

	###################################################################################################

	options['delta'] = delta
	options['gamma'] = gamma
	options['theta'] = theta
	options['vega'] = vega
	options['rho'] = rho

	options['vanna'] = vanna
	options['vomma'] = vomma
	options['charm'] = charm
	options['veta'] = veta
	options['speed'] = speed
	options['zomma'] = zomma
	options['color'] = color
	options['ultima'] = ultima

	###################################################################################################

	options.loc[:, greek_cols] = options[greek_cols].replace([-np.inf, np.inf], np.nan)
	options.loc[:, greek_cols] = options[greek_cols].round(6).fillna(0)
	options = options.sort_values(["date_current", "option_type"], ascending=True)

	return options

In [8]:
options = calculate_greeks(options)

/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in multiply
/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in multiply
/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in true_divide
/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in multiply
/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_

In [11]:
options.to_csv("data/tsla_options.csv", index=False)